In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##  Relativistic rabbit ##

The "classic" pursuit problem with a relativistic twist: A dog is chasing a (relativistic) rabbit. Its strategy is to always head directly towards where it sees the rabbit to be.

Let the rabbit and dog's speeds be w and v respectively, and work in an intertial frame with time t.

In [ ]:
def rabbit(t,w=0.8,a=1.0):
    """Returns the position of the rabbit at t."""
    return( (a,w*t) )
    #

In [ ]:
def find_tau(x,y,t):
    """Numerically solve for tau given the dog's position and t."""
    # Use bisection -- a little clunky but fast enough for us.
    taulo,tauhi = -100.0,t+1.
    xi,eta = rabbit(taulo)
    flo    = (x-xi)**2+(y-eta)**2 - (t-taulo)**2
    xi,eta = rabbit(tauhi)
    fhi    = (x-xi)**2+(y-eta)**2 - (t-tauhi)**2
    while tauhi-taulo>1e-4:
        taumid = (taulo + tauhi)/2.0
        xi,eta = rabbit(taumid)
        fmid   = (x-xi)**2+(y-eta)**2 - (t-taumid)**2
        if flo*fmid<0:
            tauhi,fhi = taumid,fmid
        else:
            taulo,flo = taumid,fmid
    return(taumid)
    #

In [ ]:
def derivs(x,y,t,dt,v=0.975):
    """Solve for the velocity of the dog."""
    tau    = find_tau(x,y,t)
    xi,eta = rabbit(tau)
    if np.abs(x-xi)<1e-6:
        # Motion is now vertical.
        dxdt = 0.0
        dydt = v
    else:
        # Get ratios for dy/dx, starting with dx=1.
        dydt = (y-eta)/(x-xi)
        # Now normalize to length v.
        scale      = v/np.sqrt(1.0+dydt**2)
        dxdt,dydt  = scale,dydt*scale
    return( (dxdt,dydt,tau) )
    #

In [ ]:
# Now plot a path.
fig,ax = plt.subplots(1,1,figsize=(10,6))
# We could solve the ODEs using one of the built-in solvers, but we're just
# going to use Euler's method for now and take really tiny steps.  Highly
# inefficient, but ...
t,dt,x,y  = 0.,1e-4,-1.0,10.0
xold,yold = x,y
#
while t<12:
    dxdt,dydt,tau = derivs(x,y,t,dt)
    # Update x and y.
    x += dxdt * dt
    y += dydt * dt
    t += dt
    if np.abs(10*t - np.rint(10*t))<9*dt:
        ax.plot([x],[y],'ko',mfc='None')
        eta,xi = rabbit(t)
        ax.plot([eta],[xi],'bs',mfc='None')
        eta,xi = rabbit(find_tau(x,y,t))
        ax.plot([eta],[xi],'cd',mfc='None')
        xold,yold = x,y
ax.plot([-5],[0],'ko',label='Dog')
ax.plot([-5],[0],'bs',label='True rabbit')
ax.plot([-5],[0],'cd',label='App. rabbit')
ax.set_xlim(-1.1,1.1)
ax.set_ylim(-50.,15.)
ax.set_xlabel(r'$x$',fontsize=18)
ax.set_ylabel(r'$y$',fontsize=18)
ax.legend()

The classical pursuit problem has an interesting history, with contributions to the mathematics going as far back as Newton. There was a lot of development in the 18th century, funded by various navies, that helped inspire developments in DEs. Prominent names associated with the problem are Bouguer and Boole.